In [3]:
import random as rand
import numpy as np
from copy import deepcopy

#Constant c used in calculation for UCT, weight between exploration and exploitation
uct_constant = 0.8

#Create a class for nodes, with parents, board, player and children. As well as values for calculating UCT
class Node:
    def __init__(self,parent,board,player):
        self.parent = parent
        self.board = board
        self.player = player
        self.children = []
        self.q = 0
        self.n = 0
        self.uct = 0

#Method for creating an empty board for 3x3 tic tac toe
def create_board():
    return [0] * 9

#Method for creating an empty tree with board as the root
def create_tree(board):
    return [Node(parent = None, board = board, player = -1)]

#Check if the current state is a terminal state
def check_terminal(node):
    board = node.board
    for i in range(3):
        if board[i * 3] == board[i * 3 + 1] == board[i * 3 + 2] != 0: return board[i]
        if board[i] == board[i + 3] == board[i + 6] != 0: return board[i]
    if board[0] == board[4] == board[8] != 0: return board[0]
    if board[2] == board[4] == board[6] != 0: return board[2]
    if all(val != 0 for val in board): return 0
    else: return 10

#Selection method. Policy is to select the node with the highest UCT. 
def selection(tree):
    unexplored_nodes = (node for node in tree if not node.children)
    selected_node = max(unexplored_nodes, key=lambda x: x.uct)
    return selected_node


#Tic tac toe action for inserting mark in the selected square
def action(node, tree, i, player):
    if node.board[i] == 0:
        board = deepcopy(node.board)
        board[i] = player
        # Create a new node with the modified board and switch players
        node1 = Node(parent=node, board=board, player=-player)
        node.children.append(node1)
        tree.append(node1)
        node1.parent = node

#Method expansion, expands all children to the selected node
def expansion(node,tree):
    if check_terminal(node) != 10:
        return node
    
    else:
        #Expand all possible children
        for i in range(0,len(node.board)):
            action(node,tree,i,node.player)

        #Select the first child that has not been selected before
        for node_temp in node.children:
            if node_temp.n == 0:
                simulation_node = node_temp
                return simulation_node

#Method for simulation
def simulation(node):
    tempNode = deepcopy(node)
    
    #While a terminal state is not reached
    while check_terminal(tempNode) == 10:
        
        #Pick random square
        i = rand.randint(0, 8)
        
        #If square i empty and current player X, insert X in square i and switch player to O. 
        if tempNode.board[i] == 0:
            tempNode.board[i] = tempNode.player
            tempNode.player = -tempNode.player
    
    #Check if the new state is a terminal node
    return check_terminal(tempNode)

#Update the value for uct
def update_uct(node):
    node.uct = (node.q/node.n) + uct_constant * np.sqrt(np.log(node.parent.n)/node.n)

#Method for backpropagataion
def backpropagation(node, value):
    n, q = 1, value

    # Update all parent nodes up to root node
    while node.parent is not None:
        node = node.parent
        n += 1
        q += value

        uct = q / n + uct_constant * np.sqrt(np.log(node.parent.n) / n)
        node.n += 1
        node.q += value
        node.uct = uct

    # Update root node
    node.n += 1
    node.q += value
    node.uct = node.q / node.n

#Calculate the best action from a certain state. Based on the highest UCT value of all children nodes
def best_action(node):
    best_child = max(node.children, key=lambda x: x.uct)
    return best_child.board

#Run 300 iterations of the 4 steps in the MCT algorithm on a board. Returns the best action from the current state.
def play(board):
    tree = create_tree(board)
    for i in range(100):
        selected_node = selection(tree)
        expansion_node = expansion(selected_node, tree)
        value = simulation(expansion_node)
        backpropagation(expansion_node,value)
    return best_action(tree[0])

#Method for printing the board.
def printBoard(board):
    print(board[0] + '|' + board[1] + '|' + board[2])
    print('-+-+-')
    print(board[3] + '|' + board[4] + '|' + board[5])
    print('-+-+-')
    print(board[6] + '|' + board[7] + '|' + board[8])
    print()
    print('----------------------------------------')
    print()

def printBoard(board):
        print("-------------")
        for i in range(3):
            print(f"| {board[i*3]} | {board[i*3+1]} | {board[i*3+2]} |")
            print("-------------")

#Method for initializing a board
def start_game():
    board = create_board()
    printBoard(board)
    while check_terminal(board) == 10:
        board = play(board)
        printBoard(board)

#Method for making moves, input a square and board and the square in that board gets updated with a X.    
def make_move(i,board):
    if board[i] == ' ':
        board[i] = 'X'
    else: 
        print('Invalid move')
        printBoard(board)
        return board
    if check_terminal(create_tree(board)[0]) == 1:
        print("X wins")
        printBoard(board)
        return board
    if check_terminal(create_tree(board)[0]) == -1:
        print("O wins")
        printBoard(board)
        return board
    if check_terminal(create_tree(board)[0]) == 0:
        print("Draw")
        printBoard(board)
        return board
    new_board = play(board)
    printBoard(new_board)
    return new_board

board = start_game()
board = make_move(1,board)

board = make_move(4,board)

board = make_move(7,board)


-------------
| 0 | 0 | 0 |
-------------
| 0 | 0 | 0 |
-------------
| 0 | 0 | 0 |
-------------


AttributeError: 'list' object has no attribute 'board'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1262dda2-abb7-4af7-a1b6-72164064af5a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>